<a href="https://colab.research.google.com/github/Busola181/SeqModel-Projects-/blob/main/letter_level_Text_Generation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch.nn as nn
import torch
import numpy as np

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
DATA_PATH="/content/drive/MyDrive/Deep_Learning/textGeneration/shakespeare.txt"
# DATA_PATH="/content/drive/MyDrive/shakespeare.txt"

In [ ]:
with open(DATA_PATH, 'r') as f:
    content = f.read()

chars = sorted(list(set(content)))

In [ ]:
idx_to_char = {i: char for i, char in enumerate(chars)}
char_to_idx = {char: i for i, char in enumerate(chars)}

In [ ]:
def create_dataset(seq_length=100, data=""):
    inputs = []
    targets = []
    for i in range(len(content)-seq_length):
        inputs.append([char_to_idx[char] for char in data[i:i+seq_length]])
        targets.append([char_to_idx[char] for char in data[i+seq_length]])
    return inputs, targets

inputs, targets = create_dataset(data=content)

In [ ]:
class TextGenNetwork(nn.Module):
    def __init__(self):
        super(TextGenNetwork, self).__init__()

        self.embedding = nn.Embedding(num_embeddings=len(chars), embedding_dim=64)
        self.lstm = nn.LSTM(input_size=64, hidden_size=512, num_layers=2, batch_first=True)
        self.fc = nn.Linear(in_features=512, out_features=len(chars))

    def forward(self, x, hidden=None, return_hidden=False):

        x = self.embedding(x)
        out, hidden = self.lstm(x, hidden)
        out = self.fc(out[:, -1, :])

        if return_hidden:
            return out, hidden
        return out

    def init_hidden(self, batch_size):
        return (torch.zeros(2, batch_size, 512).to(next(self.parameters()).device),
                torch.zeros(2, batch_size, 512).to(next(self.parameters()).device))


network = TextGenNetwork()
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
network = network.to(device)
network

TextGenNetwork(
  (embedding): Embedding(91, 64)
  (lstm): LSTM(64, 512, num_layers=2, batch_first=True)
  (fc): Linear(in_features=512, out_features=91, bias=True)
)

In [ ]:
print(len(targets))

5458837


In [ ]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(network.parameters(), lr=0.01)

batch_size = 2048
n_epochs = 10

In [ ]:
def train(model, inputs, targets, batch_size, n_epochs, seq_length=100):
    model.train()  # Set the model to training mode

    for epoch in range(n_epochs):
        hidden = model.init_hidden(batch_size)  # Initialize hidden state

        for i in range(0, len(inputs), batch_size):
            x_batch = torch.tensor(inputs[i:i + batch_size]).to(device)
            y_batch = torch.tensor(targets[i:i + batch_size]).to(device)

            hidden = None

            optimizer.zero_grad()

            output, hidden = model(x_batch, hidden, return_hidden=True)

            loss = criterion(output.view(-1, len(chars)), y_batch.view(-1))

            loss.backward()
            optimizer.step()
            torch.save(model.state_dict(), '/content/drive/MyDrive/Deep_Learning/textGeneration/parameters.pth')

        print(f'Epoch: {epoch+1}/{n_epochs}, Loss: {loss.item()}')

In [ ]:
train(network, inputs, targets, batch_size, n_epochs)

In [ ]:
def generate_text(model, start_str, n_chars, temperature=1.0):
    model.eval()

    input_seq = torch.tensor([char_to_idx[char] for char in start_str], dtype=torch.long).unsqueeze(0).to(device)

    generated_text = start_str
    hidden = None

    for _ in range(n_chars):
        output, hidden = model(input_seq, hidden, return_hidden=True)

        output = output / temperature
        probs = torch.softmax(output, dim=1).detach().cpu().numpy()

        next_char_idx = np.random.choice(len(chars), p=probs[0])
        next_char = idx_to_char[next_char_idx]

        generated_text += next_char

        input_seq = torch.tensor([[next_char_idx]], dtype=torch.long).to(device)

    return generated_text

start_str = "Once upon a "
generated_text = generate_text(network, start_str, 1000, temperature=0.8)
print(generated_text)


Once upon a posest,
  Of virting fair him all on see,
  And that I music forth thy to ecall by th' usine with thy see of the beauty please that a most thou dead nertate becaty or my the reasure,
  
  Aourse be than My few where,
  Thou art not do me?
  And it of siuse, or the is beauty not flowery be that silved,
  And my love prispire, is thou disperked,
  But doth love how though the sull my lovered by apperingings,
  And your but ending hold it to love not stept tence bestrain my must panness bear
  Surry of teals think day,
  That there have ever seasure,
    I thy collow thou so cold stand in on forth,
  Expause that thou love,
  To served my not she, and is the it as I Jutter lains,
  Which my heaven for that thou art the can withour's morn a that you had has vine there,
  But a fece of thy be me my toon
    Their panse eyes thy poinged of thee no better be which do sune thou where self,
  Sir Juls, by thou it fire.


                                                              